## Import libs

In [2]:
#Import libs 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statistics as st
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
import random
import mlflow
import mlflow.sklearn
import geopandas as gpd
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 18)

## Upload Data

In [31]:
df = pd.read_excel(r"C:\jupyter\SPP\input\sensitivity_MC_TL_bal8sand.xlsx", sheet_name='Sheet1')
unique_column_values_raw = df['well_name'].unique()
unique_column_values = [x for x in unique_column_values_raw[~pd.isnull(unique_column_values_raw)] if x!= 'well_name']
tables = {}
for value in unique_column_values:
    table_name = value
    table_df = df[df['well_name'] == value].reset_index(drop=True)
    tables[table_name] = table_df

p10 = pd.DataFrame()
p50 = pd.DataFrame()
p90 = pd.DataFrame()
for table in tables.values():
    if len(table):
        p10 = pd.concat([p10, table[table['Flag Name']=='RES'][table['Stat Name']=='P 10']])
        p50 = pd.concat([p50, table[table['Flag Name']=='RES'][table['Stat Name']=='P 50']])
        p90 = pd.concat([p90, table[table['Flag Name']=='RES'][table['Stat Name']=='P 90']])
p10.reset_index(drop=True, inplace=True)
p50.reset_index(drop=True, inplace=True)
p90.reset_index(drop=True, inplace=True)
p10.iloc[:,3:-1] = p10.iloc[:,3:-1].astype('float')
p50.iloc[:,3:-1] = p50.iloc[:,3:-1].astype('float')
p90.iloc[:,3:-1] = p90.iloc[:,3:-1].astype('float')
p10.replace(-9999, np.nan, inplace=True)
p50.replace(-9999, np.nan, inplace=True)
p90.replace(-9999, np.nan, inplace=True)
df_uncert = pd.concat([p10,p50,p90], ignore_index=True)
df_uncert = df_uncert[[ 'well_name', 'Zone Name', 'Flag Name', 'Stat Name', 'Top (M)', 'Bottom (M)',
                        'Gross (M)', 'Net(M)', 'Net to gross', 'POR avg', 'VSH avg',
                        'SW avg', 'PORV(M)', 'HCPV(M)', 'LPERM arithmetic avg',
                        'LPERM geometric avg', 'LPERM Hamonic avg']]
df_uncert.sort_values(by='well_name', inplace=True)

In [29]:
#Loading metadata, distribution wells per Platforms and all the that.
path = 'C:\\jupyter\\SPP\\input\\'
metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
metadata = metadata_init.copy()
metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
metadata.Status = metadata.Status.str.strip()
metadata.Status = metadata.Status.str.lower()
metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'
#Read data from parquet
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v6.parquet.gzip')
df_prq.rename(columns={'wellName':'well'}, inplace=True)
df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
# print('wells in df totally:', len(df_prq.well.unique()))
# Filter data with bad_well_list 
bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP']
df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
#Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
df_prq.loc[df_prq.Fluidcode_mod>0, 'gross_pay'] = 1
df_prq.loc[df_prq.Fluidcode_mod<=0, 'gross_pay'] = 0
df_prq.gross_pay = df_prq.gross_pay.astype('int')
#Getting XY coords of Balakhany formation tops
xy_coord = df_prq[['well', 'FORMATION', 'X', 'Y']]
xy_coord = xy_coord.groupby(['well', 'FORMATION']).apply(lambda x: x.iloc[0]).drop(columns=['well', 'FORMATION']).reset_index()
xy_coord = xy_coord[xy_coord.FORMATION.str.contains('Balakhany') & (xy_coord.X>0) & (xy_coord.Y>0)]
#Find top TVD_SCS for each formation
df_prq_tvdss = df_prq[['well','DEPTH','FORMATION','TVD_SCS']].groupby(['well','FORMATION']).apply(lambda x: x.iloc[0])
df_prq_tvdss = df_prq_tvdss.drop(['well','FORMATION'], axis=1).reset_index()
df_prq_tvdss = df_prq_tvdss[df_prq_tvdss.TVD_SCS>0]

In [30]:
#Calculation of TST-thickness for ALL Balakhany FU
df_fu_tst = df_prq[(df_prq.FORMATION.str.contains('Balakhany VIII')) | (df_prq.FORMATION.str.contains('Balakhany X'))]
df_fu_tst = df_fu_tst[['well', 'DEPTH','FORMATION','TST']]
df_fu_tst_top = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[0]).reset_index()
df_fu_tst_top.rename(columns={'TST':'TST_top'}, inplace=True)
df_fu_tst_bot = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1]).reset_index()
df_fu_tst_bot.rename(columns={'TST':'TST_bot'}, inplace=True)
df_fu_tst_final = df_fu_tst_top.set_index(['well','FORMATION']).join(df_fu_tst_bot.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final['TST_interv'] = round((df_fu_tst_final.TST_bot - df_fu_tst_final.TST_top),0)
df_fu_tst_final = df_fu_tst_final.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
df_fu_tst_final = df_fu_tst_final[(df_fu_tst_final.TST_interv > 0)]

In [ ]:
df_uncert.head(3)

In [ ]:
df_fu_tst_final.head(3)

In [ ]:
# Histograms tst_interv and md_gross_uncert
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=df_fu_tst_final.TST_interv, 
              xbins=dict(start=0, end=150, size=3), marker_color='blue', name='tst_interv'))
fig.add_trace(go.Histogram(
              x=df_uncert['Gross (M)'], 
              xbins=dict(start=0, end=150, size=3), marker_color='yellow', name='md_uncert'))
fig.update_traces(opacity=0.66)
fig.update_layout(title_text='Histograms tst_interv and md_gross_uncert',
                  xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='overlay')
fig.update_xaxes(nticks=40, showgrid=True)

In [105]:
df_uncert_sel = df_uncert[(df_uncert['Gross (M)'] < 45) & (df_uncert['Net(M)'] > 0)]
df_fu_tst_final_sel = df_fu_tst_final[df_fu_tst_final.TST_interv < 45]

In [ ]:
# Histograms tst_interv and md_gross_uncert
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=df_fu_tst_final_sel.TST_interv, 
              xbins=dict(start=0, end=100, size=1), marker_color='blue', name='tst_interv'))
fig.add_trace(go.Histogram(
              x=df_uncert_sel['Gross (M)'], 
              xbins=dict(start=0, end=100, size=1), marker_color='yellow', name='md_uncert'))
fig.update_traces(opacity=0.66)
fig.update_layout(title_text='Histograms tst_interv and md_gross_uncert',
                  xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                  autosize=True, width=700, height=400, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='overlay')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
df_uncert_sel['Kavg_ar_Net'] = (df_uncert_sel['Net(M)'] * df_uncert_sel['LPERM arithmetic avg']).round(0)
df_uncert_sel.loc[df_uncert_sel['Stat Name'] == 'P 10', 'NewProb'] = 'P90'
df_uncert_sel.loc[df_uncert_sel['Stat Name'] == 'P 50', 'NewProb'] = 'P50'
df_uncert_sel.loc[df_uncert_sel['Stat Name'] == 'P 90', 'NewProb'] = 'P10'
df_uncert_sel.sort_values(by=['well_name', 'NewProb'], inplace=True)
df_uncert_sel.head(6)

In [ ]:
df_uncert_sel_p10 = df_uncert_sel[(df_uncert_sel.NewProb=='P10') & (df_uncert_sel.Kavg_ar_Net>100)]
df_uncert_sel_p10.Kavg_ar_Net.quantile(0.5, interpolation="nearest")

In [ ]:
df_uncert_sel_p10.sort_values(by=['Kavg_ar_Net'], inplace=True, ascending=False)
fig = px.bar(df_uncert_sel_p10, x="well_name", y="Kavg_ar_Net")
fig.update_layout(title_text='Distribution P10 Kavg_ar_Net based on uncertainty analize with tolerance 10% for Phit, Vsh and cut-offs',
                  xaxis_title_text='wells', yaxis_title_text='Kavg_ar_Net',
                  autosize=True, width=1200, height=400, margin=dict(l=10,r=10,b=10,t=40))
fig.show()

In [ ]:
df_uncert_sel[[ 'well_name', 'Zone Name', 'Flag Name','NewProb', 'Gross (M)', 'Net(M)', 'POR avg',
                'VSH avg', 'LPERM arithmetic avg','LPERM geometric avg', 'LPERM Hamonic avg', 'Kavg_ar_Net']].sort_values(by=['well_name', 'NewProb']).head(6)


In [ ]:
df_uncert_sel.sort_values(by=['Kavg_ar_Net'], inplace=True, ascending=False)
fig = px.bar(df_uncert_sel[df_uncert_sel.Kavg_ar_Net>100], x="well_name", y="Kavg_ar_Net", color="NewProb")
fig.update_layout(title_text='Distribution P10-50-90 Kavg_ar_Net based on uncertainty analize with tolerance 10% for Phit, Vsh and cut-offs',
                  xaxis_title_text='wells', yaxis_title_text='Kavg_ar_Net',
                  autosize=True, width=1200, height=400, margin=dict(l=10,r=10,b=10,t=40))
fig.show()